#Setiment analysis of bitcoin with tweets

In [1]:
import numpy as np
import pandas as pd
from numpy import *
import cvxpy as cp
from math import log, exp
import numpy as np
from numpy.random import random
import time
#import cvxopt, cvxopt.solvers
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
sns.set(style="whitegrid")

In [2]:
!pip install pandas OpenBlender scikit-learn

     |████████████████████████████████| 60 kB 3.3 MB/s 
     |████████████████████████████████| 251 kB 10.6 MB/s 
  Created wheel for OpenBlender: filename=OpenBlender-2.11-py3-none-any.whl size=6654 sha256=204f6a977fbc668a4e2a1a98b9a0e687b5d2895ac9f46b60a5987e807c1ab621
  Stored in directory: /root/.cache/pip/wheels/55/ab/44/d9bd09091ae990c19555441e9baf3fc7794b330fa89c90f3a8
Successfully built OpenBlender


In [3]:
!pip install fsspec

     |████████████████████████████████| 132 kB 5.4 MB/s 


# the preprocessing codes and vectorizing are already suggested by open blender and can be found at the references below:
##https://www.openblender.io/#/dataset/blend/5d4c3af79516290b01c83f51/ts

##https://towardsdatascience.com/predict-any-cryptocurrency-applying-nlp-with-global-news-e938af6f7922


In [11]:
import pandas as pd
import numpy as np
import OpenBlender
import json
token = '61ab6ed9951629523799cc8bDZRKyA5X51hyJGbuNh0W4MUi2WAJre'
action = 'API_getObservationsFromDataset'

  
parameters = { 
    'token' : token,
    'id_dataset' : '5d4c3af79516290b01c83f51',
    'date_filter':{"start_date" : "2010-01-01",
                   "end_date" : "2021-08-29"} 
		#'aggregate_in_time_interval':{"time_interval_size":86400,"output":"avg","empty_intervals":"impute"} 
}
df = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']), convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df.reset_index(drop=True, inplace=True)
df['date'] = [OpenBlender.unixToDate(ts, timezone = 'GMT') for ts in df.timestamp]
df = df.drop('timestamp', axis = 1)

Task ID: '61ab6f0b0895fafb4a9d8d4b'.
Total estimated consumption: 500.28 processing units.
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 % completed.


In [12]:
df.size

19677

In [13]:
y1=np.log(df['open'])
y2=np.log(df['price'])
df['difference'] = y2-y1 


/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [14]:
df['target'] = [1 if difference > 0 else 0 for difference in df['difference']]

In [15]:
format = '%d-%m-%Y %H:%M:%S'
timezone = 'GMT'
df['timestamp'] = OpenBlender.dateToUnix(df['date'], 
                                           date_format = format, 
                                           timezone = timezone)
df = df[['timestamp', 'date', 'price', 'target']]
df.head()

,timestamp,date,price,target
0,1.629652e+09,22-08-2021 17:00:00,49264.0,1
1,1.620925e+09,13-05-2021 17:00:00,49747.0,1
2,1.620839e+09,12-05-2021 17:00:00,49725.0,0
3,1.620752e+09,11-05-2021 17:00:00,56715.0,1
4,1.620666e+09,10-05-2021 17:00:00,55851.0,0


In [16]:
search_keyword = 'bitcoin'
df = df.sort_values('timestamp').reset_index(drop = True)
OpenBlender.searchTimeBlends(token,
                             df.timestamp,
                             search_keyword)

[{'description': 'Daily prices and volumes for the digital currency BitcoinsCash.',
  'features': ['close',
   'high',
   'low',
   'market_cap',
   'open',
   'timestamp',
   'volume'],
  'id_dataset': '5d9f899b951629329a748593',
  'intersection': 47.264299203655526,
  'name': 'Bitcoins Cash Historical data',
  'num_observations': 1580,
  'url': 'https://www.openblender.io/#/dataset/explore/5d9f899b951629329a748593'},
 {'description': '',
  'features': ['date',
   'feargreed_index',
   'feargreed_classification',
   'bitcoin_search',
   'coinbase_search',
   'crypto_search',
   'ethereum_search'],
  'id_dataset': '6114e0ed9516295907e7f5d4',
  'intersection': 41.16295347393722,
  'name': 'BTC Fear&Greed   Google Analyst weekly searches',
  'num_observations': 1288,
  'url': 'https://www.openblender.io/#/dataset/explore/6114e0ed9516295907e7f5d4'},
 {'description': '',
  'features': ['settimana', 'bitcoin', 'coinbase', 'crypto', 'ethereum'],
  'id_dataset': '6114d7009516295906ef5474',
  

In [17]:
pos = {'name' : 'positive', 
                   'match_ngrams': ['positive', 'buy', 
                                    'bull', 'boost']}
blend_source = {
                'id_dataset':'5ea2039095162936337156c9',
                'feature' : 'text',
                'filter_text' : pos
            }
df11= OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df=pd.concat([df, df11.loc[:, df11.columns != 'timestamp']], axis = 1)



Task ID: '61ab6f110895fafb4a9d8d4c'.
Total estimated consumption: 9795.2 processing units.
Task confirmed. Starting download..
36.0%
71.0%
100%


In [18]:

neg = {'name' : 'negative', 
                   'match_ngrams': ['negative', 'loss', 'drop', 'plummet', 'sell', 'fundraising']}
blend_source = {
                'id_dataset':'5ea2039095162936337156c9',
                'feature' : 'text',
                'filter_text' : neg
            }
df22 = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df=pd.concat([df, df22.loc[:, df22.columns != 'timestamp']], axis = 1)

Task ID: '61ab6f1b0895fafb4a9d8d4c'.
Total estimated consumption: 9795.2 processing units.
Task confirmed. Starting download..
36.0%
71.0%
100%


In [19]:
df.head(2500)

,timestamp,date,price,target,BITCOIN_NE.text_COUNT_last1days:positive,BITCOIN_NE.text_last1days:positive,BITCOIN_NE.text_last1days:negative,BITCOIN_NE.text_COUNT_last1days:negative
0,1.357056e+09,01-01-2013 16:00:00,13.3,0,0,[],[],0
1,1.357142e+09,02-01-2013 16:00:00,13.3,0,0,[],[],0
2,1.357229e+09,03-01-2013 16:00:00,13.4,0,0,[],[],0
3,1.357315e+09,04-01-2013 16:00:00,13.5,0,0,[],[],0
4,1.357402e+09,05-01-2013 16:00:00,13.4,0,0,[],[],0
...,...,...,...,...,...,...,...,...
2495,1.587661e+09,23-04-2020 17:00:00,7495.2,1,1,[facebook buys stake in reliance jio how the ...,[],0
2496,1.587748e+09,24-04-2020 17:00:00,7506.8,1,0,[],[],0
2497,1.587834e+09,25-04-2020 17:00:00,7538.5,1,0,[],[],0
2498,1.587920e+09,26-04-2020 17:00:00,7697.1,1,0,[],[],0


In [20]:
features = ['target', 'BITCOIN_NE.text_COUNT_last1days:positive', 'BITCOIN_NE.text_COUNT_last1days:negative']
df[features].corr()['target']

target                                      1.000000
BITCOIN_NE.text_COUNT_last1days:positive    0.031512
BITCOIN_NE.text_COUNT_last1days:negative    0.013848
Name: target, dtype: float64

In [21]:

blend_source = { 
                'id_textVectorizer':'5f739fe7951629649472e167'
               }
df33 = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw') .add_prefix('VEC.')
df1 = pd.concat([df, df33.loc[:, df33.columns != 'timestamp']], axis = 1)


Task ID: '61ab6f270895fafb4a9d8d4c'.
Total estimated consumption: 9796.9 processing units.
Task confirmed. Starting download..
12.0%
25.0%
37.0%
50.0%
62.0%
75.0%
87.0%
100.0%
100%


In [22]:
X = df1.loc[:, df1.columns == 'target'].select_dtypes(include=[np.number]).values

In [23]:

correlation = df1.corr().abs()
Higher = correlation.where(np.triu(np.ones(correlation.shape), k=1).astype(np.bool))
for column in Higher.columns:
  if any(Higher[column] > 0.5):
    df1.drop(column, axis=1, inplace=True)
X = df1.loc[:, df1.columns == 'target'].select_dtypes(include=[np.number]).values
y = df1.loc[:,['target']].values
D = int(round(len(X) * 0.2))
X_train = X[:D]
y_train = y[:D]
X_test = X[D:]
y_test = y[D:]

##NEURAL NETWORK

In [24]:
class neuralnetwork():
    
    def __init__(self,X,Y,X_test,Y_test,Layer,NL):
    
        self.loss_training = list()
        self.accuracy_train = list()
        self.loss_testing = list()
        self.accuracy_testing = list()
        self.X = np.concatenate((X,np.ones((X.shape[0],1))),axis=1)
        self.Y = np.squeeze(np.eye(10)[Y.astype(np.int).reshape(-1)])
        self.X_test = np.concatenate((X_test,np.ones((X_test.shape[0],1))),axis=1)
        self.Y_test = np.squeeze(np.eye(10)[Y_test.astype(np.int).reshape(-1)])
        self.Layer = Layer
        self.NL = NL
        self.ns = self.X.shape[0]
        self.layersizes =np.array([self.X.shape[1]]+[NL]*Layer+[self.Y.shape[1]]) 
        self.initialize_weights()
        self.metrics=[self.loss_training,self.accuracy_testing]
    def sigmoid(self,x):
        AF1=1./(1.+np.exp(-x))
        return AF1
    
    def softmax(self,x):
      dd = np.exp(x)
      AF2=dd/dd.sum(axis=1,keepdims=True)
      return AF2
  
    def LOSS(self,y_pred,y):
        dd=(-np.log(y_pred))*y
        Loss=(dd).sum(axis=1).mean()
        return Loss
    
    def accuracy(self,predicted_y,y):  
        ACCURACY=np.all(predicted_y==y,axis=1).mean()
        return ACCURACY

    
    def sigmoid_derivation(self,K):
        P=K*(1-K)
        return P
    
    def categorical(self,x):  
        cat = np.zeros((x.shape[0],self.Y.shape[1]))
        catego=cat[np.arange(x.shape[0]),x.argmax(axis=1)] = 1
        return catego
    
    def initialize_weights(self):
        self.weights = list()
        for i in range(self.layersizes.shape[0]-1):
            self.weights.append(np.random.uniform(-1,1,size=[self.layersizes[i],self.layersizes[i+1]]))
        self.weights = np.asarray(self.weights)
    
    def initialize_layer(self,batch_s):
        self.hidden = [np.empty((batch_s,layer)) for layer in self.layersizes]
    
    def feedforward(self,batch):
        hiddenlayer = batch
        self.hidden[0] = hiddenlayer
        for i,weights in enumerate(self.weights):
            hiddenlayer = self.sigmoid(hiddenlayer.dot(weights))
            self.hidden[i+1]=hiddenlayer
        self.o1 = self.softmax(self.hidden[-1])
    
    def backprop(self,y_batch):
        DT = (self.o1 - y_batch)*self.sigmoid_derivation(self.hidden[-1])
        for i in range(1,len(self.weights)+1):
            self.weights[-i]-=self.learning_rate*(self.hidden[-i-1].T.dot(DT))/self.batch_s
            DT = self.sigmoid_derivation(self.hidden[-i-1])*(DT.dot(self.weights[-i].T))
            
    def prediction_xx(self,X):
        x_new=np.concatenate((X,np.ones((X.shape[0],1))),axis=1)
        self.initialize_layer(x_new.shape[0])
        self.feedforward(x_new)
        output1=self.categorical(self.o1)
        return output1

    
    def evaluating(self,X,Y):

        predicted_x  = self.prediction_xx(X)
        ACC=self.accuracy(predicted_x ,Y)
        return ACC
        
    def training(self,batch_s=8,epochs=25,learning_rate=1.0):
        self.learning_rate = learning_rate
        self.batch_s=batch_s

        for epoch in range(epochs):
            start = time.time()
            
            self.initialize_layer(self.batch_s)
            shuffle = np.random.permutation(self.ns)
            loss_training = 0
            accuracy_train = 0
            x_batches = np.array_split(self.X[shuffle],self.ns/self.batch_s)
            y_batches = np.array_split(self.Y[shuffle],self.ns/self.batch_s)
            for batch_x,batch_y in zip(x_batches,y_batches):
                self.feedforward(batch_x)  
                loss_training += self.LOSS(self.o1,batch_y)
                accuracy_train += self.accuracy(self.categorical(self.o1),batch_y)
                self.backprop(batch_y)
                
            loss_training = (loss_training/len(x_batches))
            accuracy_train = (accuracy_train/len(x_batches))
            self.loss_training.append(loss_training)
            self.accuracy_train.append(accuracy_train)

            self.initialize_layer(self.X_test.shape[0])
            self.feedforward(self.X_test)
            loss_testing = self.LOSS(self.o1,self.Y_test)
            accuracy_testing = self.accuracy(self.categorical(self.o1),self.Y_test)
            self.loss_testing.append(loss_testing)
            self.accuracy_testing.append(accuracy_testing)


In [25]:
model = neuralnetwork(X_train,y_train,X_test,y_test,15,128)
model.training(batch_s=8,epochs=1,learning_rate=0.01)
y_pred = model.prediction_xx(X_test)

print('Accuracy Score:')
print(model.evaluating(X_test,y_test))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Accuracy Score:
0.5251222765673633


##SUPPORT VECTOR MACHINE

In [29]:

from sys import exit

class SVM():  
    def initialization(self):
            self.params = None
            self.support_vector_machine = None
            self.std = None
            self.kernel_p=None
    def kernel(self, x1, x2):
            covariance=x1@x2.T
            return covariance
    def normalization(self, x, set_value=False):
            if set_value: self.std = x.std(axis=0) 
            norm=x/self.std       
            return norm
    def train(self, x, y, c=1, eta=0.1, kernel_p=None):
        samples,features=x.shape
        
        y = np.copy(y)
        x = self.normalization(x, set_value=True)
        y = y.reshape([samples])  
        Kernel = self.kernel(x, x)   
        P = cp.Parameter(shape=Kernel.shape, value=Kernel, PSD=True)
        P1 = np.ones(samples)
        G = np.vstack((np.diag(np.ones(samples) * -1), np.identity(samples)))
        h = np.hstack((np.zeros(samples), np.ones(samples) * c))
        V1 = cp.Variable(samples)
        V2 = cp.Variable(samples)
        d = V1-V2
        TT=V1+V2
        prob = cp.Problem( cp.Minimize((1/2)*cp.quad_form(d, P) - y.T@d + eta*P1.T@(TT) ),
                 [G@V1 <= h, G@V2 <= h, P1.T@d == 0.0])
        prob.solve()
        L  = np.ravel(d.value)
        mask = np.abs(L) > 1e-5
        L[np.invert(mask)] = 0
        SVM_x = x[mask]
        SVM_y = y[mask]
        SVM_K = self.kernel(SVM_x, SVM_x)
        L = L.reshape([-1, 1])
        inter = np.mean(SVM_y - SVM_K.T@(L[mask]))
        self.params = {'lagranges' : L[mask],'intercept':inter}
        self.support_vector_machine = {'SVM_x': SVM_x}
        
        
    def predict(self, X):
        X = self.normalization(X)
        L = self.params['lagranges']
        inter = self.params['intercept']
        SVM_x = self.support_vector_machine['SVM_x']
        Kernel=self.kernel(SVM_x, X)
        y=Kernel.T@L + inter
        yy=y.reshape([-1])
        return yy


In [50]:
model=SVM()
model.train(X_train,y_train,1000)
y_pred = model.predict(X_test)
df_res = pd.DataFrame({'y_test':y_test[:, 0], 'y_pred':y_pred})
threshold = 0.5
preds = [1 if val > threshold else 0 for val in df_res['y_pred']]
print(accuracy_score(preds, df_res['y_test']))

1.0


##NAIVE BAYESIAN

In [37]:
import numpy as np
     
class NaiveBayesian:
    
    def training(self, X, Y):
       
    
        self.label, count = np.unique(y, return_counts=True)
        for x in X.T:
          dd=np.unique(x)
          self.x_classes = np.array(dd)
      
        self.phi_y = 1 * count/count.sum()
        for k in self.label:
          dd=X[Y==k].mean(axis=0)
          self.u = np.array(dd)
       
        for k in self.label:
          out3=X[Y==k].var(axis=0)+1
          self.var_x = np.array(out3)
        
        return self

    def predict(self, X):
        predicted=np.apply_along_axis(lambda x: self.compute_prob(x), 1, X)
        return predicted
    
    def compute_prob(self, x):
        AA=self.label[np.argmax(np.array(probs(self, x, Y)) for y in range(len(self.label)))]
        return 
    
    def probs(self, x, Y):
        dd=self.var_x[Y]
        R = 1/np.sqrt(2* np.pi * (dd))
        o=(2 * self.var_x[Y])
        out1=np.exp(-1* np.square(x - self.u[Y]) /o )
        y1=np.prod(R *out1 )
        return y1

    def evaluate(self, x, Y):
        predicted=self.predict(x)
        ACC=(predicted == Y).mean()
        return ACC

In [38]:
clf = NaiveBayesian().training(X_train, y_train)
y_pred = clf.predict(X_test)

print('Accuracy Score:')
print(clf.evaluate(X_test, df_res['y_test']))


Accuracy Score:
0.0
